## Explorer tool for survey data on Australian Slang

This notebook allows you to run complex queries on the data collected in our survey.

In a Jupyter notebook, the code in a cell is executed by placing your cursor in the relevant cell and then either:
- clicking on the button in the menu bar above (the one that looks like a **Play** button)
- pressing `Ctrl+Enter`
- pressing `Shift+Enter`

In this notebook, you need to run the first three code cells at the start of a session, but then all the necessary tools and data are loaded and you will not need to run those cells again.

Just closing this window does not end a SWAN session. For good housekeeping (i.e. releasing resources back to the system), you should end the session by selection **Shutdown Current Session** from the **File** menu.

In [ ]:
# load packages
# this line not needed in SWAN but may be needed elsewhere
# install.packages('stringr')
library(stringr)

In [ ]:
# ranges for queries
# table for retrieving data by prompt
q_col <- seq(from = 15, to = 80, by = 5)
start <- q_col + 1
end <- q_col + 4
prompt_cols <- data.frame(q_col, start, end)
prompt_cols[14,2] <- 0
prompt_cols[14,3] <- 0

# table for retrieving data by state
states <- c('NT','NSW','VIC','QLD','SA','WA','TAS')
pc_start <- c(799,1999,2999,3999,4999,5999,6999)
pc_end <- c(1000,3000,4000,5000,6000,7000,8000)
pc_states <- data.frame(states, pc_start, pc_end)

In [ ]:
# read in data
slang_data <- read.csv('NotebookData_postcode.csv', header = TRUE, stringsAsFactors = FALSE)

#inspect to check it worked
slang_data[1:5,]


## Constructing a query

We can choose values for gender, age, state of residence and the range of prompts of interest. This means you can, for example, retrieve responses to the prompt about alcohol for women over the age of 60 living in Victoria. The relevant values are set out below; make your choices and then type the corresponding values in the following code cell to set the value of the variables which will be fed to the query. 

You do not have to select a value for each variable, but if you are not using any of them, the value for Gender,Age or State should be `= ''` (i.e. with no space between the inverted commas, and for Prompt the value should be `0`.

Currently, only one prompt can be selected; an upgrade to allow for multiple prompts to be selected is coming.

If you want to change the variables in your query, you have to amend the cell below and then run it again. You can then rerun the query code; doing this will overwrite any existing results.

**Gender**: Woman, Man, Non-binary / gender diverse, Prefer not to say

**Age**: Under 18, 19 - 40, 40 - 60, Over 60

**State**: NT, NSW, VIC, QLD, SA, WA, TAS

**Prompts**: 1 = very good, 2 = very bad, 3 = stupid person, 4 = attractive person, 5 = attractive person, 6 = attractive male, 7 = unattractive person, 8 = arrogant, 9 = nonsense, 10 = alcohol, 11 = intoxicated, 12 = person who does not contribute, 13 = part of the body

In [ ]:
#set query variables
gender <- 'Woman'
age <- '19 - 40'
state <- 'NSW'

# one prompt only
prompts <- 1 

In [ ]:
# construct query and run it

message <- c('Your query returns', '0', 'rows')
# get postcodes and run query
if (state != '') {
    state_codes <- subset(pc_states, pc_states$states == state)
    result_1 <- subset(slang_data, eval(expression(slang_data$postcode > state_codes$pc_start & slang_data$postcode < state_codes$pc_end)))
    } else {result_1 <- slang_data}

# filter by gender
if (gender != '') {
    result_2 <- subset(result_1, result_1$Q11 == gender)
} else {result_2 <- result_1}

# filter by age
if (age != '') {
    result_3 <- subset(result_2, result_2$Q3 == age)
} else {result_3 <- result_2}

if (nrow(result_3) == 0) {
    message_str <- paste(message, collapse = ' ')
    print(message_str)
} else {
    message[2] <- nrow(result_3)
    message_str <- paste(message, collapse = ' ')
    print(message_str)
    if (prompts > 0) {
# delete columns for unwanted prompts
        result_final <- result_3[c(1:14, prompt_cols[prompts,2]:prompt_cols[prompts,3])]
    } else {result_final <- result_3}
   if (nrow(result_final) < 5) {
       result_final
       } else {
       result_final[1:5 ,]
       }
}
  


## Exporting results

You can export your results as a comma-separated values file which can be opened by Excel (or other spreadsheet software). The syntax for doing this is `write.csv([name of table to export], '[location for saving].csv')`. If you just provide a file name for the export (and it has to include the .csv extension!), the file will be saved in your current working directory in Cloudstor or SWAN. It is easy to download the file from there to save on your computer. But you can also specify the location on your computer directly. If you do this, you have to provide a complete path to the preferred location, and this should use / as dividers (not \\).

In [ ]:
write.csv(result_final, 'export.csv' )